Cài thêm thư viện

In [ ]:
!pip install fasttext
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=3127972 sha256=7f35729f8861a1e9cd519dd8129cc90d5864e079960c9b484c6c699170f3fa0f
  Stored in directory: /root/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.3/591.3 KB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.0 MB/s eta 0:00:00


Tải model vietnamese từ fastext

In [ ]:
!gdown https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vi.300.bin.gz

Downloading...
From: https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vi.300.bin.gz
To: /content/cc.vi.300.bin.gz
100% 4.50G/4.50G [01:38<00:00, 45.6MB/s]


Tải stop-word để tiền xử lý

In [ ]:
!gdown https://drive.google.com/uc?id=12D06zcJzNi5BepCpTDsyE0MdwVCKmsy8&export=download

Downloading...
From: https://drive.google.com/uc?id=12D06zcJzNi5BepCpTDsyE0MdwVCKmsy8
To: /content/vietnamese-stopwords.txt
100% 20.5k/20.5k [00:00<00:00, 9.56MB/s]


In [ ]:
!gunzip /content/cc.vi.300.bin.gz

Tải data từ 300 bài báo đã được cào sẵn về.

In [ ]:
!gdown https://drive.google.com/uc?id=1ZAC3mLjoWprkvRTR61xmtBn9OseorpCh&export=download

Downloading...
From: https://drive.google.com/uc?id=1ZAC3mLjoWprkvRTR61xmtBn9OseorpCh
To: /content/newpaper.csv
100% 1.29M/1.29M [00:00<00:00, 102MB/s]


Import thư viện

In [ ]:
import fasttext
import fasttext.util
model = fasttext.load_model('/content/cc.vi.300.bin')
model.get_dimension()

300

Đọc dữ liệu đầu vào để so sánh

In [ ]:
import pandas as pd
data = pd.read_csv("/content/newpaper.csv")

base_document = data['Document'][0]
documents = data['Document'][:]


In [ ]:
import pandas as pd
data = pd.read_excel("/content/Book1.xlsx")

base_document = data['Document'][0]
documents = data['Document'][:]


In [ ]:
model.get_nearest_neighbors('tuyệt_vời')

[(0.7256906628608704, 'hoàn_hảo'),
 (0.7149642705917358, 'tuyệt_hảo'),
 (0.7065508365631104, 'tuyệt_diệu'),
 (0.6789242029190063, 'tuyêt_vời'),
 (0.6640727519989014, 'tuyệt'),
 (0.6614848375320435, 'thú_vị'),
 (0.6341797709465027, 'tốt'),
 (0.632213294506073, 'Tuyệt_vời'),
 (0.6304551362991333, 'tuyệt_vờ'),
 (0.6303631067276001, 'lý_tưởng')]

In [ ]:
from scipy import spatial
import pandas as pd
import numpy as np

#Hàm tính độ tương đồng theo cosine
def cosin_similarity(vector1, vector2):
  return np.dot(vector1,vector2)/(np.linalg.norm(vector1)*np.linalg.norm(vector2))

In [ ]:
import string
from underthesea import word_tokenize

def stopwords():
    sw = []
    with open("/content/vietnamese-stopwords.txt", encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        sw.append(line.replace("\n",""))
    return sw


def preprocess(text):
	# Steps:
	# 1. lowercase
	# 2. Lammetize. (It does not stem. Try to preserve structure not to overwrap with potential acronym).
	# 3. Remove stop words.
	# 4. Remove punctuations.
	# 5. Remove character with the length size of 1.

	lowered = str.lower(str(text))

	stop_words = set(stopwords())
	word_tokens = word_tokenize(lowered)

	words = []
	for w in word_tokens:
		if w not in stop_words:
			if w not in string.punctuation:
				if len(w) > 1:
					words.append(w)

	return words


In [ ]:
def generateVector(sentence):
    return model.get_sentence_vector(sentence)

In [ ]:
def word2vec_similarity(doc1, doc2):
  doc1_process = str(preprocess(doc1))
  doc2_process = str(preprocess(doc2))
  doc1_vec = generateVector(doc1_process)
  doc2_vec = generateVector(doc2_process)
  scores =  cosin_similarity(doc1_vec,doc2_vec)

  return scores

In [ ]:
result = []
for i, document in enumerate(documents):
  scores = word2vec_similarity(base_document, documents[i])
  result.append({'index': i, 'score': scores})

	#Sort scores in decreasing order
  result = sorted(result, key=lambda x: x['score'], reverse=True)
print(list(result))

[{'index': 0, 'score': 1.0000001}, {'index': 2, 'score': 0.99999696}, {'index': 1, 'score': 0.9998529}, {'index': 3, 'score': 0.98695534}]
